Transformations

Select required columns
chaning the data type
Adding calculated colums
renaming columns
Filtering
String and Date Formattinigs
Aggregations
remove nulls and duplicates
Joins
unions
rank, dense rank, getting lastrow and next row values


In [0]:
#create a datafram from a static list
# syntax: spark.createDataFrame(data = <list>, schema= <schemalist)

staticlist = [("    James", "Smith", "USA", "",5000),
              ("Michael", "Rose", "Canada", "NY",6000),
              ("Robert", "", "USA", "CA",7000),
              ("Maria", "Anne", "New York", "FL",8000),
              ("Jen", "Mary", "USA", "CA",9000),
              ("Sara", "Miller", "UK", "LND",10000),
              ("robert", "Bevone","UK","MRC",2500),
              ("james", "Bond", "UK", "LND",7500),
              ("MARIA", "Bond", "UK", "LND",6500)
              ]

columns = ["firstname", "lastname", "country", "state", "salary"]
df = spark.createDataFrame(data = staticlist, schema =columns)
df.show()


+---------+--------+--------+-----+------+
|firstname|lastname| country|state|salary|
+---------+--------+--------+-----+------+
|    James|   Smith|     USA|     |  5000|
|  Michael|    Rose|  Canada|   NY|  6000|
|   Robert|        |     USA|   CA|  7000|
|    Maria|    Anne|New York|   FL|  8000|
|      Jen|    Mary|     USA|   CA|  9000|
|     Sara|  Miller|      UK|  LND| 10000|
|   robert|  Bevone|      UK|  MRC|  2500|
|    james|    Bond|      UK|  LND|  7500|
|    MARIA|    Bond|      UK|  LND|  6500|
+---------+--------+--------+-----+------+



In [0]:
#check the datatype of any variable
type(df)

pyspark.sql.connect.dataframe.DataFrame

In [0]:
#check the schema of the datafram
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)



In [0]:
# craete a new schma for a dataframe using the StructType and StructField classes+
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html

from pyspark.sql.types import *
structSchema = StructType([
    StructField("firstname", StringType(), False),
    StructField("lastname", StringType(), True),
    StructField("country", StringType(), False),
    StructField("state", StringType(), False),
    StructField("salary", StringType(), False)
])


In [0]:
df = spark.createDataFrame(data = staticlist, schema = structSchema)
df.show()
#check the schema of the datafram
df.printSchema()

+---------+--------+--------+-----+------+
|firstname|lastname| country|state|salary|
+---------+--------+--------+-----+------+
|    James|   Smith|     USA|     |  5000|
|  Michael|    Rose|  Canada|   NY|  6000|
|   Robert|        |     USA|   CA|  7000|
|    Maria|    Anne|New York|   FL|  8000|
|      Jen|    Mary|     USA|   CA|  9000|
|     Sara|  Miller|      UK|  LND| 10000|
|   robert|  Bevone|      UK|  MRC|  2500|
|    james|    Bond|      UK|  LND|  7500|
|    MARIA|    Bond|      UK|  LND|  6500|
+---------+--------+--------+-----+------+

root
 |-- firstname: string (nullable = false)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = false)
 |-- state: string (nullable = false)
 |-- salary: string (nullable = false)



In [0]:
# most used pyspark libaries

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from delta.tables import *

In [0]:
# SELECT SPECIFIC COLUMNS FROM A DATAFRAME USING THE "COLUMN NAME" SELECTION
# USING name in inverted coluumns is not case sensitive

df_emp_fist_last = df.select("firstname", "LASTNAME")
df_emp_fist_last.show()


+---------+--------+
|firstname|LASTNAME|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|        |
|    Maria|    Anne|
|      Jen|    Mary|
|     Sara|  Miller|
|   robert|  Bevone|
|    james|    Bond|
|    MARIA|    Bond|
+---------+--------+



In [0]:
# OPTION 2: USING the dataframe method
df_2 = df.select(df.firstname, df.lastname)
df_2.show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|        |
|    Maria|    Anne|
|      Jen|    Mary|
|     Sara|  Miller|
|   robert|  Bevone|
|    james|    Bond|
|    MARIA|    Bond|
+---------+--------+



In [0]:
#option3: using datafram method 2
# this method the column name ar not case sensitive

df_3 = df.select(df["firstname"], df["LASTNAME"])
df_3.show()

+---------+--------+
|firstname|LASTNAME|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|        |
|    Maria|    Anne|
|      Jen|    Mary|
|     Sara|  Miller|
|   robert|  Bevone|
|    james|    Bond|
|    MARIA|    Bond|
+---------+--------+



In [0]:
# select the columns based on column method
# column name is not case sensitive in col mehtod

df_4 = df.select(col("firstname"), col("LASTNAME"))
df_4.show()

+---------+--------+
|firstname|LASTNAME|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|        |
|    Maria|    Anne|
|      Jen|    Mary|
|     Sara|  Miller|
|   robert|  Bevone|
|    james|    Bond|
|    MARIA|    Bond|
+---------+--------+



In [0]:
# select the column names based on whildcard name
df5 = df.select(df.colRegex("`.*name*`"))
df5.show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|        |
|    Maria|    Anne|
|      Jen|    Mary|
|     Sara|  Miller|
|   robert|  Bevone|
|    james|    Bond|
|    MARIA|    Bond|
+---------+--------+



In [0]:
#changing the datatype of the column
#option1: using the select method

df_cast1 = df.select("firstname", "lastname", "country", "state",
                      df.salary.cast(IntegerType()))
df_cast1.printSchema()

root
 |-- firstname: string (nullable = false)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = false)
 |-- state: string (nullable = false)
 |-- salary: integer (nullable = true)



In [0]:
#option2: using the withcolumn method
df_cast2 = df.withColumn("salary", df.salary.cast(IntegerType()))
df_cast2.printSchema()

root
 |-- firstname: string (nullable = false)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = false)
 |-- state: string (nullable = false)
 |-- salary: integer (nullable = true)



In [0]:
df_cast2 = df_cast2.withColumn("country",df.country.cast(IntegerType()))
df_cast2.show()

---------------------------------------------------------------------------
NumberFormatException                     Traceback (most recent call last)
File <command-8113337319912522>, line 2
      1 df_cast2 = df_cast2.withColumn("country",df.country.cast(IntegerType()))
----> 2 df_cast2.show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1217, in DataFrame.show(self, n, truncate, vertical)
   1216 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1217     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:958, in DataFrame._show_string(self, n, truncate, vertical)
    941     except ValueError:
    942         raise PySparkTypeError(
    943             errorClass="NOT_BOOL",
    944             messageParameters={
   (...)
    947             },
    948         )
    950 table, _ = DataFrame(
    951     plan.S

In [0]:
# Adding a new calculated columns using the with column method
# add a new salay column with double the current salary
df_7 =df.withColumn("salary_new", df.salary*2)
df_7.show()

+---------+--------+--------+-----+------+----------+
|firstname|lastname| country|state|salary|salary_new|
+---------+--------+--------+-----+------+----------+
|    James|   Smith|     USA|     |  5000|     10000|
|  Michael|    Rose|  Canada|   NY|  6000|     12000|
|   Robert|        |     USA|   CA|  7000|     14000|
|    Maria|    Anne|New York|   FL|  8000|     16000|
|      Jen|    Mary|     USA|   CA|  9000|     18000|
|     Sara|  Miller|      UK|  LND| 10000|     20000|
|   robert|  Bevone|      UK|  MRC|  2500|      5000|
|    james|    Bond|      UK|  LND|  7500|     15000|
|    MARIA|    Bond|      UK|  LND|  6500|     13000|
+---------+--------+--------+-----+------+----------+



In [0]:
# add a new column full name, concatenate firstname and lastname
df_7 = df_7.withColumn("FullName",concat(df.firstname,lit(" "),df.lastname))
df_7.show()

                       

+---------+--------+--------+-----+------+----------+---------------+
|firstname|lastname| country|state|salary|salary_new|       FullName|
+---------+--------+--------+-----+------+----------+---------------+
|    James|   Smith|     USA|     |  5000|     10000|    James Smith|
|  Michael|    Rose|  Canada|   NY|  6000|     12000|   Michael Rose|
|   Robert|        |     USA|   CA|  7000|     14000|        Robert |
|    Maria|    Anne|New York|   FL|  8000|     16000|     Maria Anne|
|      Jen|    Mary|     USA|   CA|  9000|     18000|       Jen Mary|
|     Sara|  Miller|      UK|  LND| 10000|     20000|    Sara Miller|
|   robert|  Bevone|      UK|  MRC|  2500|      5000|  robert Bevone|
|    james|    Bond|      UK|  LND|  7500|     15000|     james Bond|
|    MARIA|    Bond|      UK|  LND|  6500|     13000|     MARIA Bond|
+---------+--------+--------+-----+------+----------+---------------+



In [0]:
# add a static column called ReportName and values hould be "empinfo"
df_8 = df.withColumn("ReportName",lit("empinfo"))
df_8.show()

+---------+--------+--------+-----+------+----------+
|firstname|lastname| country|state|salary|ReportName|
+---------+--------+--------+-----+------+----------+
|    James|   Smith|     USA|     |  5000|   empinfo|
|  Michael|    Rose|  Canada|   NY|  6000|   empinfo|
|   Robert|        |     USA|   CA|  7000|   empinfo|
|    Maria|    Anne|New York|   FL|  8000|   empinfo|
|      Jen|    Mary|     USA|   CA|  9000|   empinfo|
|     Sara|  Miller|      UK|  LND| 10000|   empinfo|
|   robert|  Bevone|      UK|  MRC|  2500|   empinfo|
|    james|    Bond|      UK|  LND|  7500|   empinfo|
|    MARIA|    Bond|      UK|  LND|  6500|   empinfo|
+---------+--------+--------+-----+------+----------+

